### Problem Statement
Export a model defined in PyTorch to ONNX and then import the ONNX model into TensorFlow to run it.

### Install the libraries

In [0]:
pip install torchvision torch

In [0]:
pip install onnx

     |████████████████████████████████| 4.8MB 702kB/s 


In [0]:
pip install onnx_tf

     |████████████████████████████████| 153kB 8.4MB/s 


### Define the model

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Define the constants

In [0]:
batch_size=64
test_batch_size=1000
epochs=60
lr=0.01
momentum=0.5
no_cuda=False
seed=1
log_interval=300

### Define the train and test methods

In [0]:
import torch.optim as optim
from torchvision import datasets, transforms

def train(model, device, train_loader, epoch):
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        
        output = model(data)
        loss = F.nll_loss(output, target)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test(model, device, test_loader):
    
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        
        for data, target in test_loader:
            
            data, target = data.to(device), target.to(device)
            
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    

### Set the training environment

In [0]:

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### Load the dataset

In [0]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


### Train the model

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [0]:
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.329947
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.775774
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.519010
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.318953

Test set: Average loss: 0.2121, Accuracy: 9407/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.452491
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.287368
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.338704
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.317470

Test set: Average loss: 0.1236, Accuracy: 9615/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.299001
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.516145
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.249795
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.454217

Test set: Average loss: 0.1002, Accuracy: 9685/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.409993
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.164142
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.201001
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.172851

Test set: Average loss: 0.0845, Accu

### Save the model

In [0]:
# Save the trained model to a file
torch.save(model.state_dict(), 'mnist.pth')

### Export the model to ONNX

In [0]:
from torch.autograd import Variable

# Load the trained model from file
trained_model = Net()
trained_model.load_state_dict(torch.load('mnist.pth'))

# Export the trained model to ONNX
dummy_input = Variable(torch.randn(1, 1, 28, 28)) # one black and white 28 x 28 picture will be the input to the model
torch.onnx.export(trained_model, dummy_input, "mnist.onnx")

### Load the model to TensorFlow

In [0]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX file
model = onnx.load('mnist.onnx')

# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.6/dist-packages/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


In [0]:
#Export backend representation to a Tensorflow proto file
tf_rep.export_graph('mnist.pb')